In [1]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.graph_objs as go

class ADCPlotter(object):
    def __init__(self, mode='inline', port=8050, width='100%', height='650px', color_scale='Viridis'):
        self.mode = mode
        self.port = port
        self.width = width
        self.height = height
        self.image = None
        self.hybridx = None
        self.hybridy = None
        self.terminate = False
        self.params = {
            'colorScale': color_scale,
            'error': ''
        }
        self.heatmap = {
            'data': [go.Heatmap(
                z = None
            )]
        }
        self.bar_x = {
            'data': [go.Bar(
                x = None,
                y = None
            )]
        }
        self.bar_y = {
            'data': [go.Bar(
                x = None,
                y = None
            )]
        }
        self.app = JupyterDash()
        self.app.layout = html.Div([
            html.Div(
                style = {'display': 'flex'},
                children = [
                    dcc.Graph(
                        id = 'bar-y-graph',
                        figure = self.bar_y,
                        config={'displayModeBar': False}
                    ),
                    dcc.Graph(
                        id = 'heatmap-graph',
                        figure = self.heatmap,
                        config={'displayModeBar': False}
                    )
                ]
            ),
            dcc.Graph(
                id = 'bar-x-graph',
                figure = self.bar_x,
                config={'displayModeBar': False}
            ),
            dcc.Store(
                id = 'image-store',
                data = self.image
            ),
            dcc.Store(
                id = 'hybrid-x-store',
                data = self.image
            ),
            dcc.Store(
                id = 'hybrid-y-store',
                data = self.image
            ),
            dcc.Store(
                id = 'params-store',
                data = self.params
            ),
            dcc.Interval(
                id = 'update-store',
                interval = 100,
                n_intervals = 0
            )
        ])
        self.app.clientside_callback(
            """
            function(image, hybridX, hybridY, params) {
              const errorOutput = [
                {
                  layout: {
                    xaxis: {visible: false},
                    yaxis: {visible: false}
                  }
                }, {
                  layout: {
                    xaxis: {visible: false},
                    yaxis: {visible: false}
                  }
                }, {
                  layout: {
                    title: {
                      text: 'No valid ADC data found',
                      font: {color: 'red'},
                      xref: 'x',
                      x: 0,
                    },
                    xaxis: {visible: false},
                    yaxis: {visible: false}
                  }
                }
              ];

              if (params.error !== '') {
                const errorMessage = params.error;
                errorOutput[2].layout.title.text = errorMessage;
                return errorOutput;
              }

              if (!image || !hybridX || !hybridY) {
                const errorMessage = 'No valid ADC data found';
                errorOutput[2].layout.title.text = errorMessage;
                return errorOutput;
              }

              const PLOT_HEIGHT = 300;
              const BAR_SIZE = 60;

              const font_color = '#777777';
              const plot_bgcolor = 'rgba(0.75, 0.75, 0.75, 0.1)';
              const paper_bgcolor = 'rgba(0, 0, 0, 0)';

              const numRows = image.length;
              const numCols = image[0].length;

              let height = PLOT_HEIGHT;
              let width = Math.floor(height * (numCols / numRows));

              const barSize = BAR_SIZE;

              const barYLMargin = 40;
              const barYRMargin = 40;
              const barYTMargin = 20;
              const barYBMargin = 30;
              const barYWidth = barSize + barYLMargin + barYRMargin;
              const barYHeight = height + barYTMargin + barYBMargin;

              const heatLMargin = 0;
              const heatRMargin = 110;
              const heatTMargin = barYTMargin;
              const heatBMargin = barYBMargin;
              const heatWidth = width + heatLMargin + heatRMargin;
              const heatHeight = height + heatTMargin + heatBMargin;

              const barXLMargin = barYWidth + heatLMargin;
              const barXRMargin = 110;
              const barXTMargin = 10;
              const barXBMargin = 10;
              const barXWidth = width + barXLMargin + barXRMargin;
              const barXHeight = barSize + barXTMargin + barXBMargin;

              const minRow = image.map((row) => {
                return Math.min.apply(Math, row);
              });
              const minZ = Math.min.apply(Math, minRow);
              const maxRow = image.map((row) => {
                return Math.max.apply(Math, row);
              });
              const maxZ = Math.max.apply(Math, maxRow);
              const heatmapFigure = {
                data: [{
                  z: image,
                  zmin: minZ,
                  zmax: maxZ,
                  type: 'heatmap',
                  showscale: true,
                  colorscale: params.colorScale,
                  colorbar: {
                    tickformat: '<-d',
                    tickmode: 'array',
                    tickvals: [minZ, maxZ]
                  }
                }],
                layout: {
                  width: heatWidth,
                  height: heatHeight,
                  margin: {
                    l: heatLMargin,
                    r: heatRMargin,
                    t: heatTMargin,
                    b: heatBMargin
                  },
                  paper_bgcolor,
                  font: {
                    color: font_color
                  },
                  xaxis: {
                    ticks: '',
                    showticklabels: false
                  },
                  yaxis: {
                    ticks: '',
                    showticklabels: false
                  }
                }
              };

              const barXMin = Math.min.apply(Math, hybridX);
              const barXMax = Math.max.apply(Math, hybridX);
              const barXFigure = {
                data: [{
                  y: hybridX,
                  type: 'bar',
                  width: 0.5
                }],
                layout: {
                  width: barXWidth,
                  height: barXHeight,
                  margin: {
                    l: barXLMargin,
                    r: barXRMargin,
                    t: barXTMargin,
                    b: barXBMargin
                  },
                  plot_bgcolor,
                  paper_bgcolor,
                  font: {
                    color: font_color
                  },
                  xaxis: {
                    mirror: true,
                    showline: true,
                    showgrid: false,
                    ticks: '',
                    tickformat: '>-d',
                    tickmode: 'array',
                    tickvals: [],
                    linecolor: '#A9A9A9'
                  },
                  yaxis: {
                    mirror: true,
                    showline: true,
                    showgrid: false,
                    ticks: '',
                    tickformat: '>-d',
                    tickmode: 'array',
                    tickvals: [barXMin, barXMax],
                    range: [barXMin, barXMax],
                    linecolor: '#A9A9A9',
                    zerolinecolor: '#A9A9A9'
                  }
                }
              };

              const barYMin = Math.min.apply(Math, hybridY);
              const barYMax = Math.max.apply(Math, hybridY);
              const barYFigure = {
                data: [{
                  x: hybridY,
                  type: 'bar',
                  width: 0.5
                }],
                layout: {
                  width: barYWidth,
                  height: barYHeight,
                  margin: {
                    l: barYLMargin,
                    r: barYRMargin,
                    t: barYTMargin,
                    b: barYBMargin
                  },
                  plot_bgcolor,
                  paper_bgcolor,
                  font: {
                    color: font_color
                  },
                  xaxis: {
                    side: 'top',
                    mirror: true,
                    showline: true,
                    showgrid: false,
                    ticks: '',
                    tickformat: '>-d',
                    tickmode: 'array',
                    tickvals: [barYMin, barYMax],
                    range: [barYMin, barYMax],
                    linecolor: '#A9A9A9',
                    zerolinecolor: '#A9A9A9'
                  },
                  yaxis: {
                    mirror: true,
                    showline: true,
                    showgrid: false,
                    ticks: '',
                    tickformat: '>-d',
                    tickmode: 'array',
                    tickvals: [],
                    linecolor: '#A9A9A9'
                  }
                }
              };

              return [heatmapFigure, barXFigure, barYFigure];
            }
            """,
            [Output('heatmap-graph', 'figure'),
            Output('bar-x-graph', 'figure'),
            Output('bar-y-graph', 'figure')],
            Input('image-store', 'data'),
            Input('hybrid-x-store', 'data'),
            Input('hybrid-y-store', 'data'),
            Input('params-store', 'data')
        )
        self.app.callback(
            [Output('image-store', 'data'),
            Output('hybrid-x-store', 'data'),
            Output('hybrid-y-store', 'data'),
            Output('params-store', 'data'),
            Output('update-store', 'disabled')],
            Input('update-store', 'n_intervals')
            )(self.__update_image_store_data)

    def __update_image_store_data(self, n_intervals):
        return [self.image, self.hybrid_x, self.hybrid_y, self.params, self.terminate]

    def run_plot(self):
        self.app.run_server(mode = self.mode, host = 'localhost', port = self.port, width = self.width, height = self.height)

    def update_plot(self, report):
        if ('delta' in report or 'raw' in report or 'baseline' in report):
            self.image = report[1]['image']
            self.hybrid_x = report[1]['hybridx']
            self.hybrid_y = report[1]['hybridy']

    def show_error(self, error):
        self.params['error'] = error

    def stop_plot(self):
        self.terminate = True
